# Scalarization testing


### ASF

Now there is standard Achievement Scalarizing Function implemented, so let's use it.

In [1]:
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
import logging

In [17]:
logger = logging.getLogger('pyomo.opt')
logger.setLevel(logging.ERROR)

In [3]:
revenue, carbon, deadwood, ha = init_boreal()
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('glpk')

In [4]:
ind = 100
x = np.dstack((new_normalize(nan_to_bau(revenue[:ind]).values),
               new_normalize(nan_to_bau(carbon[:ind]).values),
               new_normalize(nan_to_bau(deadwood[:ind]).values),
               new_normalize(nan_to_bau(ha[:ind]).values)))
X = np.dstack((nan_to_bau(revenue[:ind]).values,
               nan_to_bau(carbon[:ind]).values,
               nan_to_bau(deadwood[:ind]).values,
               nan_to_bau(ha[:ind]).values))

In [5]:
%%time
ref = np.array((ide[0], 0, 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [6]:
%%time
ref = np.array((0, ide[1], 0, 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model,X), axis=0))

In [7]:
%%time
ref = np.array((0, 0, ide[2], 0))
asf = ASF(ide, nad, ref, X)
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

### Timing

The ASF function seems to work properly so no problem. Let's see how much time does it take when running in full speed.

In [9]:
%%time
n_revenue = nan_to_bau(revenue)
n_carbon = nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha= nan_to_bau(ha)
x_full = np.dstack((new_normalize(n_revenue.values),
                    new_normalize(n_carbon.values),
                    new_normalize(n_deadwood.values),
                    new_normalize(n_ha.values)))
X_full = np.dstack((n_revenue.values,
                    n_carbon.values,
                    n_deadwood.values,
                    n_ha.values))

In [10]:
%%time
ref = np.array((0, 0, 0, 0))
asf_fullx = ASF(ide, nad, ref, x_full)
opt.solve(asf_fullx.model)
print(np.sum(values_to_list(asf_fullx.model,X_full), axis=0))

Lets also compare results when using normalized data for optimization, and when letting the procedure normalize it by itself


## Comparing scalarizations

Now we have ASF, STOM ja GUESS scalarizations available. Lets compare them

In [12]:
ref_test = np.array((2,3,4,2))

In [19]:
asf = ASF(ide, nad, ref_test, x, scalarization='ASF')
opt.solve(asf.model)
print(np.sum(values_to_list(asf.model, X), axis=0))

In [20]:
stom = ASF(ide, nad, ref_test, x, scalarization='STOM')
opt.solve(stom.model)
print(np.sum(values_to_list(stom.model, X), axis=0))

In [21]:
guess = ASF(ide, nad, ref_test, x, scalarization='GUESS')
opt.solve(guess.model)
print(np.sum(values_to_list(guess.model, X), axis=0))

There truly is differences between results, so it seems that scalarization functions are working properly (?)